<a href="https://colab.research.google.com/github/zhengyanggao02/zhengyanggao_DTSC3020Fall2025/blob/main/Part_2_R3%2C_R4%2C_R5_Fall2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Generate a random graph
import networkx as nx
N = 60
p = 0.07
SEED = 1000
k = 3
f = 0.6
G = nx.gnp_random_graph(N, p, seed=SEED)
print("Connected =", nx.is_connected(G))

Connected = True


In [ ]:
# Define the length range of the edge
import random
for u, v in G.edges(): G[u][v]["weight"] = random.randint(5, 25)

In [ ]:
# Simulate traffic
from collections import Counter
import random

def simulate_od_pairs(G, T=100, seconds=36000, seed=1000, sample=1.0):
    rng = random.Random(seed)
    nodes = list(G.nodes())
    trips = int(T * seconds * sample)

    return Counter(tuple(sorted(rng.sample(nodes, 2))) for _ in range(trips))

traffic = simulate_od_pairs(G, T=100, seconds=36_000, sample=1.0)

In [ ]:
# find path
candidates = [pair for pair in traffic.keys() if not G.has_edge(*pair)]

In [ ]:
# shortest path
def sp_len(u, v):
    a, b = (u, v) if u < v else (v, u)
    return nx.astar_path_length(G, a, b, weight="weight")

In [ ]:
# compute benefits
def benefit_of(u, v, f):
    cnt = traffic.get((u, v) if u < v else (v, u), 0)
    if cnt == 0:
        return 0
    s = sp_len(u, v)
    return int((1 - f) * s * cnt)

benefits = {e: benefit_of(e[0], e[1], f) for e in candidates}

In [ ]:
# select top3
top3_R3 = sorted(benefits.items(), key=lambda x: x[1], reverse=True)[:k]
print("Recommended:")
for (u,v), b in top3_R3:
    print(f"  {(u,v)}  benefit={b}")

Recommended:
  (26, 50)  benefit=63763
  (26, 40)  benefit=63324
  (9, 29)  benefit=63304


In [ ]:
# R4：For the same image and the same traffic, only change f
f = 0.8
benefits = {e: benefit_of(e[0], e[1], f) for e in candidates}
top3_R4 = sorted(benefits.items(), key=lambda x: x[1], reverse=True)[:k]
print("Recommonded （f=0.8）:")
for (u, v), b in top3_R4:
    print(f"  {(u, v)}  benefit={b}")

Recommonded （f=0.8）:
  (26, 50)  benefit=31881
  (26, 40)  benefit=31662
  (9, 29)  benefit=31652


In [ ]:
# R5：P-doubled reconstruction image and re-simulation
p = 0.14
f = 0.6
G = nx.gnp_random_graph(N, p, seed=SEED)
print("Connected =", nx.is_connected(G))

import random
for u, v in G.edges(): G[u][v]["weight"] = random.randint(5, 25)

from collections import Counter
import random

def simulate_od_pairs(G, T=100, seconds=36000, seed=1000, sample=1.0):
    rng = random.Random(seed)
    nodes = list(G.nodes())
    trips = int(T * seconds * sample)

    return Counter(tuple(sorted(rng.sample(nodes, 2))) for _ in range(trips))

traffic = simulate_od_pairs(G, T=100, seconds=36_000, sample=1.0)

candidates = [pair for pair in traffic.keys() if not G.has_edge(*pair)]

def sp_len(u, v):
    a, b = (u, v) if u < v else (v, u)
    return nx.astar_path_length(G, a, b, weight="weight")

def benefit_of(u, v, f):
    cnt = traffic.get((u, v) if u < v else (v, u), 0)
    if cnt == 0:
        return 0
    s = sp_len(u, v)
    return int((1 - f) * s * cnt)

benefits = {e: benefit_of(e[0], e[1], f) for e in candidates}

top3_R4 = sorted(benefits.items(), key=lambda x: x[1], reverse=True)[:k]
print("Recommended（p = 0.14）:")
for (u,v), b in top3_R4:
    print(f"  {(u,v)}  benefit={b}")

Connected = True
Recommended（p = 0.14）:
  (14, 53)  benefit=42800
  (1, 50)  benefit=39014
  (50, 53)  benefit=38934
